In [1]:
using Distributions, DiffBase
using ReverseDiff: GradientTape, GradientConfig, gradient, gradient!, compile
using Turing: _hmc_step

[Turing]: AD chunk size is set as 40


In [2]:
θ_dim = 1

function lj_func(θ)
  _lj = zero(Real)
  
  s = 1

  m = θ[1]
  _lj += logpdf(Normal(0, sqrt(s)), m)

  _lj += logpdf(Normal(m, sqrt(s)), 2.0)
  _lj += logpdf(Normal(m, sqrt(s)), 2.5)

  return _lj
end

neg_lj_func(θ) = -lj_func(θ)
const f_tape = GradientTape(neg_lj_func, randn(θ_dim))
const compiled_f_tape = compile(f_tape)

function grad_func(θ)
    
  inputs = θ
  results = similar(θ)
  all_results = DiffResults.GradientResult(results)

  gradient!(all_results, compiled_f_tape, inputs)

  neg_lj = all_results.value
  grad, = all_results.derivs

  return -neg_lj, grad

end

stds = ones(θ_dim)
θ = randn(θ_dim)
lj = lj_func(θ)

chn = []
accept_num = 1

function dummy_print(args...)
  nothing
end

totla_num = 5000
for iter = 1:totla_num
    
  push!(chn, θ)
  θ, lj, is_accept, τ_valid, α = _hmc_step(θ, lj, lj_func, grad_func, 5, 0.05, stds; dprint=dummy_print)
  accept_num += is_accept

end

@show mean(chn), lj
@show accept_num / totla_num

(mean(chn), lj) = ([1.47888], -5.070614428936219)
accept_num / totla_num = 1.0002


1.0002

In [3]:
using Turing

In [4]:
@model simple_gauss() = begin
    s = 1
    m ~ Normal(0,sqrt(s))
    2.0 ~ Normal(m, sqrt(s))
    2.5 ~ Normal(m, sqrt(s))
end

mf = simple_gauss()
chn = sample(mf, HMC(2000, 0.05, 5))

println("mean of m: $(mean(chn[:m][1000:end]))")

[Turing]:  Assume - `m` is a parameter
 @~(::ANY, ::ANY) at compiler.jl:76


[HMC] Sampling...  0%  ETA: 2:33:34
  ϵ:         0.05
  α:         1.0

[HMC] Finished with
  Running time        = 5.263474241999997;
  Accept rate         = 0.999;
  #lf / sample        = 4.9975;
  #evals / sample     = 1.0005;
  pre-cond. diag mat  = [1.0].



[HMC] Sampling...100% Time: 0:00:06


mean of m: 1.504830400164147


In [12]:
using Base.Test

In [17]:
function test_grad(model_f, grad_func)
    vi = mf()
    d = length(vi.vals)
    @testset "Gradient using random inputs" begin
        for _ = 1:10000
            theta = rand(d)
            @test Turing.gradient_r(theta, vi, mf) == grad_func(theta)[2]
        end
    end
end

test_grad (generic function with 1 method)

In [18]:
test_grad(mf, grad_func)

Test Summary:                                |  Pass  Total
Unit test for gradient based on random input | 10000  10000


Base.Test.DefaultTestSet("Unit test for gradient based on random input", Any[], 10000, false)